In [11]:
#selenium e webdriver automatizam ações no navegador
#pandas permite trabalhar com planilhas no Python

from selenium import webdriver
from selenium.webdriver.common.keys import Keys #permite digitação no navegador
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait #faz o selenium aguardar até determinada condição ocorrer
from selenium.webdriver.support import expected_conditions as EC #verifica se a condição esperada é válida
import pandas as pd

In [ ]:
lista_alunos = pd.read_excel("alunos.xlsx") #cria variável que recebe a planilha do excel
lista_alunos["RA"] = lista_alunos["RA"].astype(str) #converte a coluna RA em string
print(lista_alunos.info()) #mostra informações gerais da planilha

In [ ]:
navegador = webdriver.Chrome() #inicia o navegador
navegador.get("https://omnium.agilixbuzz.com/") #acessa a página

try:
    element = WebDriverWait(navegador, 10).until(EC.presence_of_element_located((By.ID, "mat-input-0"))) #aguarda até 10s, para localizar o elemento login
except:
    print("Falha ao localizar elemento")
    pass    

for linha in range(0, 26):
    try:
        navegador.find_element_by_xpath('/html/body/app-root/app-before-login/app-login/mat-toolbar/form/div/mat-form-field[1]/div/div[1]/div/input').clear() #apaga os dados na caixa input login, evita concatenação (caso tenha algo escrito na caixa, as informações se juntam e o resultado será falha no login)
        navegador.find_element_by_xpath('/html/body/app-root/app-before-login/app-login/mat-toolbar/form/div/mat-form-field[2]/div/div[1]/div/input').clear() #apaga os dados na caixa input senha, evita concatenação
        
        navegador.find_element_by_xpath('/html/body/app-root/app-before-login/app-login/mat-toolbar/form/div/mat-form-field[1]/div/div[1]/div/input').send_keys(lista_alunos["LOGIN"][linha]) #seleciona login e insere as credenciais
        navegador.find_element_by_xpath('/html/body/app-root/app-before-login/app-login/mat-toolbar/form/div/mat-form-field[2]/div/div[1]/div/input').send_keys(lista_alunos["RA"][linha]) #seleciona senha e insere as credenciais
        navegador.find_element_by_xpath('/html/body/app-root/app-before-login/app-login/mat-toolbar/form/div/mat-form-field[1]/div/div[1]/div/input').send_keys(Keys.ENTER) #aperta a tecla Enter
        
        try:
            element = WebDriverWait(navegador, 3).until(EC.presence_of_element_located((By.XPATH, '/html/body/app-root/app-after-login/div/mat-sidenav-container/mat-sidenav-content/mat-toolbar/div[2]/button[2]'))) #aguarda até 10s, para localizar o elemento menu
        except:
            pass
      
        navegador.find_element_by_xpath('/html/body/app-root/app-after-login/div/mat-sidenav-container/mat-sidenav-content/mat-toolbar/div[2]/button[2]').click() #clica no menu
        
        lista_alunos.loc[linha, "VERIFICAÇÃO"] = "Verificado" #registra na planilha que foi verificado

    except:
        lista_alunos.loc[linha, "VERIFICAÇÃO"] = "Falha na verificação"#registra na planilha que houve falha
        navegador.find_element_by_tag_name('body').send_keys(Keys.ESCAPE) #aperta ESC para sair da mensagem de login incorreto       
        pass
           
    try:
        for div in range (1,5): #esse laço serve para encontrar o valor correto da <div> do botão sair. Esse botão fica alternando em locais diferentes
            try:
                div = str(div)
                navegador.find_element_by_xpath('/html/body/div['+ div +']/div[2]/div/div/div/button[7]').click()#sair 
            except:
                pass    
    except:
        pass

display(lista_alunos) #mostra a lista com a nova coluna "Verificação"

In [14]:
navegador.quit() #encerra o navegador
lista_alunos.to_excel("alunos verificados.xlsx", index = False) #salva as informações em uma nova planilha